In [2]:
import numpy as np
from scipy.stats import binom

In [3]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [4]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [102]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    for i in range(7):
        S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]
        #print(S, E, I, R)
        M = theta[:16].reshape(4,4)
        betas = 1/4 * np.dot(M, I)
        #print(betas)
        B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
        C = np.random.binomial(E, 1-np.exp(-h*pho)) 
        D = np.random.binomial(I, 1-np.exp(-h*gamma)) 
        S_new = S - B
        E_new = E + B - C
        I_new = I + C - D
        R_new = R - D
        X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
        #print(B, C, D)
    return X_new

In [106]:
# Likelihood of y knowing theta

def likelihood(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X_new = mutation(theta, X)
        C[i] = np.random.binomial(X_new[:, 2], 1-np.exp(-h*pho))
    return(binom.pmf(y, np.sum(C, axis=0), theta[16:]))

In [121]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    thetas = []
    w_init = np.zeros((N, 4))
    for n in range(N):
        theta = np.concatenate((np.random.gamma(5, 1, 16), np.random.normal(0.5, 0.5, 4)), axis=None)
        thetas.append(theta)
        w_init[n] = likelihood(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    for r in range(R):
        for k in range(4):
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), p=W[:, k])
                w_hat = np.ones(N)
            else:
                A = np.arange(0, N)
                w_hat = w
            for n in range(N):
                thetas[n][k:k+4] = thetas[A[n]][k:k+4]
                thetas[n][16+k] = thetas[A[n]][16+k]
                X[n][k] = X[A[n]][k]
        for n in range(N):
            w[n] = w_hat[n] * likelihood(y[r], thetas[n], X[n])
            X[n] = mutation(thetas[n], X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))

In [122]:
Y = np.random.randint(8000, size =(19,4))
BootstrapFilter(Y, 0.1, 100, R=19)